### Operator.itemgetter()

The Python standard library is full of gems. (I know, I know -- the Ruby language also has lots of gems.  Ha, ha.) One of the modules I've discovered, and started to use, over the last year is the "operator" module, which comes in handy in all sorts of places.

To understand how the "operator" module can help, remember that when we use an operator in Python, such as +, %, or even **, the operator is turned into a method call. In other words, when we say
```
2 + 3
we're really invoking
    int.__add__(2, 3)
```
What does this have to do with us? Well, let's assume that we want to create a simple calculator program using prefix notation. That is, if the user enters
    + 2 2

then we want to add two numbers. The most natural way to do this, in my opinion, is with a "dispatch table" -- that is, a table containing functions, and we can choose the best function from that table based on the user's input.  For example, we can imagine the following:
```   
def add(x,y):
    return x + y

def sub(x,y):
    return x - y

ops = {'+': add,
      '-':sub}
```     
This will definitely work! But it's kind of ridiculous that we need to define our own functions for adding and subtracting.

There are other options, of course. One of them is to use "lambda", which creates an anonymous function:
```
    ops = {'+': lambda x,y: x+y,
           '-':  lambda x,y: x-y}
```
But in many cases, using "lambda" here is overkill. I mean, I love lambda -- but a huge number of Python developers don't know what it does, and those who do think that it's some weird holdover from other languages that Python doesn't need.

The thing is, there's something to that -- because an easier, and more Pythonic, way to do what I did above is to use the "operator" module. It implements all of the operators that we know and use every day, but as functions. Thus, instead of the above implementations, I can simply say:
```
import operator
ops = {'+', operator.add,
       '-', operator.sub}
```
But that's just the start of what the "operator" module provides. Perhaps the most interesting ones are "itemgetter" and "attrgetter" which make it easy to do certain kinds of operations.

Consider, for example, if I have the following list of dicts:
```
presidents = [{'first':'George', 'last':'Washington', 'year':1788},
              {'first':'John', 'last':'Adams', 'year':1796},
             {'first':'Thomas', 'last':'Jefferson', 'year':1800},
             {'first':'James', 'last':'Madison', 'year':1808}]
```
If can easily iterate over this list and print each dict nicely:
```
for one_item in presidents:
    print("{first} {last}, started in {year}".format(**one_item))
```
(Note that here, I'm taking advantage of the fact that "format" takes **kwargs, and that I can turn a dictionary into kwargs with ** before its name during the function invocation.)

What if I want to sort this list of dicts by last name?  Now we're in a similar situation to what I had above -- I want to retrieve a particular element ("last") from each dict.  I could write a function, or I could use "lambda".  Or I could use operator.itemgetter, which does exactly this:
```
presidents.sort(key=operator.itemgetter('last'))
for one_item in presidents:
    print("{first} {last}, started in {year}".format(**one_item))
```
Note that operator.itemgetter is a function that returns a function. Which means that we use operator.itemgetter whenever we want to get a function back.  It even takes a variable number of arguments, which means that we can sort by more than one field -- for example, "last" and then "first" -- if we want.

Let's say that I want to allow my users to choose the field by which we will sort the presidents. I could do it as follows:
    sort_by = input("Enter sort key")
```
presidents.sort(key=operator.itemgetter(sort_by))
for one_item in presidents:
    print("{first} {last}, started in {year}".format(**one_item))
```



### operator.attrgetter()

What about operator.attrgetter?  That's useful if we want to get the same attribute from a variety of objects. For example, let's say that I have a few classes, and I want to know what each of them inherits from. In Python, inheritance is defined by the "__bases__" attribute, which is a tuple. I can thus say:
    get_base = operator.attrgetter('__bases__')
    get_base(str)

I can even do it to a few classes:
    class Foo(object):
        pass
```
class Bar(Foo):
    pass

for one_class in [int, str, Foo, Bar]:
    print(one_class.__name__, get_base(one_class))
```

Here's a way to get the versions of some modules in Python:

```    
get_version = operator.attrgetter('__version__')
for modname in sys.modules:
    try:
        print(modname, get_version(sys.modules[modname]))
    except AttributeError:
        pass
```   
Note that there isn't much of a standard here; some modules use "__version__", while others use "version" and still others use "VERSION".

I've found myself using "itemgetter" and "attrgetter" in a growing number of my Python programs. Is it just me, or do you also use these?  Reply, and let me know!

